# Project 4 - Cheri Hung

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

Answer: To predict the likelihood of being admitted to graduate school based on an applicant's GPA, GRE and how prestige his/her undergraduate school is ranked.

### Dataset:  Write up a description of your data and any cleaning that was completed

Answer: The data is a hypothetical dataset provided by UCLA. Data points with null values were dropped. The dataset contains 400 records with 3 records missing a value for one or more variables. Since there are only 3 out of 400 null data points, they are decidedly dropped instead of being filled in with imputed data. 

The dataset has no clear timeframe associated with this dataset. And there are no guidelines published on how the schools are evaluated into the four rankings. This dataset also do not include other demographic data (e.g. age, sex, race). Therefore, this analysis makes the assumption that those factors may have no significant impact on admission.

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 

| Not Admitted | Admitted
---| ---|---
GPA | mean(std)  | mean(std)
GRE |mean(std) | mean(std)
Prestige 1 | frequency (%) | frequency (%)
Prestige 2 | frequency (%) | frequency (%)
Prestige 3 |frequency (%) | frequency (%)
Prestige 4 |frequency (%) | frequency (%)

In [76]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

df = pd.read_csv("../assets/admissions.csv")
df = df.dropna()

def add_prefix_to_index(x):
    return 'Prestige '+str(int(x))

def calc_percentage(x, sum):
    return str(round((x / float(sum)), 4) * 100)+'%'

str_na = 'Not Admitted'
str_a = 'Admitted'
pres_index = ['Prestige 1', 'Prestige 2', 'Prestige 3' ,'Prestige 4']

#Admitted
df_admit = df[df['admit'] == 1] 
df_a_presCount = df_admit.groupby('prestige').size() #group by prestige
df_a_mean = df_admit.describe()[['gre', 'gpa']].loc['mean']

#Not Admitted
df_notadmit = df[df['admit'] == 0]
df_n_presCount = df_notadmit.groupby('prestige').size() #group by prestige
df_n_mean = df_notadmit.describe()[['gre', 'gpa']].loc['mean']

#Transform Prestige values for joined table display
df_a_presCount = df_a_presCount.rename(lambda x: add_prefix_to_index(x))
df_a_presCount = df_a_presCount.apply(lambda x: calc_percentage(x, df_a_presCount.sum()))
df_n_presCount = df_n_presCount.rename(lambda x: add_prefix_to_index(x))
df_n_presCount = df_n_presCount.apply(lambda x: calc_percentage(x, df_n_presCount.sum()))

prestige_count = pd.DataFrame({str_na: df_n_presCount, str_a: df_a_presCount}, 
                              index=pres_index)

summary_table = pd.DataFrame({'GRE':[round(df_n_mean.gre, 2), round(df_a_mean.gre, 2)], 
                              'GPA':[round(df_n_mean.gpa, 2), round(df_a_mean.gpa, 2)]}, 
             index=[str_na, str_a])

### GRE/GPA Mean and Prestige Frequency % by Admission Status

In [79]:
final_table = summary_table.join(prestige_count.transpose()).transpose()
final_table

,Not Admitted,Admitted
GPA,3.35,3.49
GRE,573.58,618.57
Prestige 1,10.33%,26.19%
Prestige 2,35.06%,42.06%
Prestige 3,34.32%,22.22%
Prestige 4,20.3%,9.52%


### Methods: Write up the methods used in your analysis

Answer: 

A **distribution analysis** was performed to determine possible correlation between GRE, GPA scores and admission rates. Both showed a non-normal distribution. Other **exploratory analysis** was performed as well to gain an overview on the dataset.

**Colinearity analysis** was then performed to determine which factors may have a correlation with the admit outcome. While no variable has an overwhelming colinearity (-1 or 1) with each other or with admit, the **presitge** factor has a comparatively higher colinearity value than GRE and GPA to admission status and has little colinearity with those two variables. This formed our initial hypothesis that Prestige can be used as a predictor for admission.

Finally, to test the hypothesis and create a possible prediction model, a **logistic regression** was performed on the **prestige** factor. Odds ratio was first hand-calculated to gain some intuition on the odds of admission based on prestige. Then, using Statsmodel, a full logistic regression was completed.

### Results: Write up your results

Answer:

Based on the logistic regression, there does appear to be a positive correlation between better the prestige and higher the odds of admission. By generating a dataset that controls the GRE and GPA input vaules, we saw that the model does indeed predict a higher odds of being admitted if an applicant is from a more prestigious school. (1=most prestigious, 4=least prestigious)

### Visuals: Provide a table or visualization of these results

These two charts show an upward trend in odds as GPA and GRE score increases. But we can clearly see that by having attended a better ranked undergraduate school (prestige of 1 or 2) dramatically increases an applicant's chance of being admitted. The starting OR is almost 0.3 points higher for applicants from ranked 1 schools than from ranked 4 schools. (Source: Unit 3 Project Analysis)

<img src='../assets/regression-chart-gpa.png' height=40% width=40%>

<img src='../assets/regression-chart-gre.png' height=40% width=40%>

By looking at a snapshot of generated modeling data, we can also observe the same conclusion. The table below shows that with the GRE and GPA values remaining the same, the predicted odds of admission (pred_admitted) increases as the prestige ranking improves. The difference between #1 and #4 prestige is notable enough to rely on this analysis. (Source: Unit 3 Project Analysis)

<img src='../assets/table-snapshot.png' height=50% width=50%>

### Discussion: Write up your discussion and future steps

Answer: 

The model developed from the UCLA dataset shows that an applicant from a lower-ranked undergraduate school faces a tougher chance of being admitted regardless of how well he/she scored on the GRE or had performed in school (i.e. GPA score). Assuming the model holds true, it is somewhat disheartening that the postive effect of a neutral and standardized evaluation factor such the GRE score is not higher.

While this model isn't terribly surprising, the postiive correlation is clear enough that can now be fine-tuned by running against a larger dataset from the real world. And see how ot still fits or holds. Given this model is based on a small set of data points, when fitting to real-world data, it is likely to be too simplified or underfitting because of that. A significant issue to consider is how to assign the prestige ranks to new data points. We do not have a guideline on how prestige were created for this UCLA dataset. So a natural next step is to either find the guideline/algorithm or to somehow establish one ourselves. And then re-train our model accordingly.